In [13]:
import json
import csv

In [14]:
devices = ["amazonEcho",
        "awairAirQuality",
        "lifxbulb",
        "nestsmokesensor",
        "pixstarphotoframe",
        "ringdoorbell",
        "tplinkcamera",
        "tribyspeaker",
        "withingssleepsensor",
        # devices that are not analyzed by savinduwannigama
        'samsungsmartcam',
        'withingsbabymonitor'
        ]

In [15]:
filePath = '../../MUD/MUD_Profiles/'
fileType = 'Mud.json'

In [16]:
# dictionaries to store devices classified by MUD flows
# dict = {"MUDFlow" : [list of devices using the MUDFlow]}
IPFlows_withPorts = {}  # stores IPV4/IPV6 flows using TCP/UDP
IPFlows_withoutPorts = {}  # stores IPV4/IPV6 flows NOT using TCP/UDP
EthFlows = {}  # stores flows NOT using IPV4/IPV6

# dictionary to store MUD flows classified by devices
# dict = {"device" : [list of MUDFlows used by the device]}
devices_MUDFlows = {}

In [17]:
# this loop runs for each MUD profile
for device in devices:

    # initializing an empty list for the device
    devices_MUDFlows[device] = []

    print('checking device: ' + device)
    f = open(filePath + device + fileType)
    data = json.load(f)

    # lists to store ACLs for the device
    from_device_acls = []
    to_device_acls = []

    # # reading the types of access lists for from-device policies  # NOT USED
    # for acl in data["ietf-mud:mud"]["from-device-policy"]["access-lists"]["access-list"]:
    #     from_device_acls.append(acl['name'])
    
    # # reading the types of access lists for to-device policies  # NOT USED
    # for acl in data["ietf-mud:mud"]["to-device-policy"]["access-lists"]["access-list"]:
    #     to_device_acls.append(acl['name'])

    # will iterate once for each  ACL
    for acl in data["ietf-access-control-list:access-lists"]["acl"]:
        flowType = acl["name"].split('-')[1]  # values: "ipv4" | "ipv6" | "ethernet"
        # flowDirection = acl["name"].split('-')[0]  # values: "from" | "to" 
        
        # handling ACLs which has ACEs with IP flows
        if flowType == "ipv4" or flowType == "ipv6":
            # will iterate once for each ACE in the ACL
            for ace in acl["aces"]["ace"]:
                protoType = ace["matches"][flowType]["protocol"]  # values: ip protocol number
                
                # TODO: "operator": "range"  (present in tplinkcameraMUD.json)

                # Extracting the port number 
                # handling a TCP flows
                if protoType == 6:
                    if "destination-port" in ace["matches"]["tcp"]:
                        if ace["matches"]["tcp"]["destination-port"]["operator"] == "eq":
                            portNum = ace["matches"]["tcp"]["destination-port"]["port"]
                            # print('1')
                        else:
                            protoType = -1
                    elif "source-port" in ace["matches"]["tcp"]:
                        if ace["matches"]["tcp"]["source-port"]["operator"] == "eq":
                            portNum = ace["matches"]["tcp"]["source-port"]["port"]
                            # print('2')
                        else:
                            protoType = -1
                # handling UDP flows
                elif protoType == 17:
                    if "destination-port" in ace["matches"]["udp"]:
                        if ace["matches"]["udp"]["destination-port"]["operator"] == "eq":
                            portNum = ace["matches"]["udp"]["destination-port"]["port"]
                            # print('3')
                    elif "source-port" in ace["matches"]["udp"]:
                        if ace["matches"]["udp"]["source-port"]["operator"] == "eq":
                            portNum = ace["matches"]["udp"]["source-port"]["port"]
                            # print('4')
                # handling other IP flows
                else:
                    portNum = '*'
                    # print('5')

                # making the key string to store the flow in the relevant dictionary
                # format example: "ipv4-6-443" (for a TCP/443 flow over ipv4)
                dictKey = flowType + '-' + str(protoType) + '-' + str(portNum)

                if protoType == -1:
                    continue
                # adding IP flows with port numbers to the list

                if protoType == 6 or protoType == 17:  # flow with port number
                    # adding a key of not already existing (empty list as value)
                    print(f'')
                    if dictKey not in IPFlows_withPorts:
                        IPFlows_withPorts[dictKey] = []
                    # adding if the device is not already in the list
                    if device not in IPFlows_withPorts[dictKey]:
                        IPFlows_withPorts[dictKey].append(device)
                # adding IP flows without port numbers to the list
                else:
                    # adding a key of not already existing (empty list as value)
                    if dictKey not in IPFlows_withoutPorts:
                        IPFlows_withoutPorts[dictKey] = []
                    # adding if the device is not already in the list
                    if device not in IPFlows_withoutPorts[dictKey]:
                        IPFlows_withoutPorts[dictKey].append(device)

                # checking if the flow is not in the list corresponding to the device in the dict devices_MUDFlows
                if dictKey not in devices_MUDFlows[device]:
                    devices_MUDFlows[device].append(dictKey)

        # handling ACLs which has ACEs with Eth flows
        elif flowType == "ethernet":
            # will iterate once for each ACE in the ACL
            for ace in acl["aces"]["ace"]:
                ethType = ace["matches"]["eth"]["ethertype"]  # values: etherType in hex
                
                # making the key string to store the flow in the relevant dictionary
                # format example: "ethernet-0x0006" 
                dictKey = flowType + '-' + str(ethType)

                # adding IP flows with port numbers to the list
                # adding a key of not already existing (empty list as value)
                if dictKey not in EthFlows:
                    EthFlows[dictKey] = []

                # adding if the device is not already in the list
                if device not in EthFlows[dictKey]:
                    EthFlows[dictKey].append(device)
                
                # checking if the flow is not in the list corresponding to the device in the dict devices_MUDFlows
                if dictKey not in devices_MUDFlows[device]:
                    devices_MUDFlows[device].append(dictKey)

print('\n\nIPFlows_withPorts:')
print(IPFlows_withPorts)
print('\n\nIPFlows_withoutPorts:')
print(IPFlows_withoutPorts)
print('\n\nEthFlows:')
print(EthFlows)
print('\n\ndevices_MUDFlows:')
print(devices_MUDFlows)
    

checking device: amazonEcho




















































checking device: awairAirQuality













checking device: lifxbulb














checking device: nestsmokesensor



































































































checking device: pixstarphotoframe












checking device: ringdoorbell















checking device: tplinkcamera





























checking device: tribyspeaker






































checking device: withingssleepsensor









checking device: samsungsmartcam


























checking device: withingsbabymonitor





















IPFlows_withPorts:
{'ipv4-6-443': ['amazonEcho', 'awairAirQuality', 'pixstarphotoframe', 'ringdoorbell', 'tplinkcamera', 'tribyspeaker', 'withingssleepsensor', 'samsungsmartcam'], 'ipv4-17-123': ['amazonEcho', 'lifxbulb', 'ringdoorbell', 'tplinkcamera', 'tribyspeaker', 'samsungsmartcam'], 'ipv4-17-1900': ['amazonEcho',

#### Mapping the MUDFlows with FDCP models

In [18]:
# loading the FDCP models
fdcpNames = [ 'arp',
        'bacnet',
        'beep',
        'bgp',
        'bittorrent',
        'cdp',
        'classic_stun',
        'cmp',
        'dcerpc_cl',
        'dcerpc_co',
        'dhcp',
        'dhcpv6',
        'dnp3',
        'dns',
        'ftp',
        'http',
        'icmp',
        'icmpv6',
        'igmp',
        'imap',
        'ipp',
        'irc',
        'isakmp',
        'kerberos',
        'ldap',
        'lldp',
        'llmnr',
        'lpd',
        'mdns',
        'nbns',
        'nntp',
        'ntp',
        'ospf',
        'pop3',
        'rdpudp',
        'rip',
        'rsh',
        'rtmp',
        'rtp',
        'rtsp',
        'sip',
        'smb',
        'smb2',
        'smtp',
        'snmp',
        'srvloc',
        'ssdp',
        'ssh',
        'stun',
        'syslog',
        'telnet',
        'tftp',
        'tibia',
        'tls',
        'whois',
        'xmpp'
        ]

all_FDCPs = []

filePath = '../../data/FDCP_models/'
fileType = '.json'



In [19]:
# loading all the FDCP models
for fdcpName in fdcpNames:
    # print('loading FDCP: ' + fdcpName)
    f = open(filePath + fdcpName + fileType)
    data = json.load(f)
    all_FDCPs.append(data)

# variable to hold the FDCP-MUDFlow mapping
MUDFlowtoFDCP = {}  # {'mudFlow': [corresponding FDCP abbreviations]}
MUDFlowWithNoFDCP = [] 
# print(all_FDCPs[0])

In [20]:
# IPFlows_withPorts = {}  # stores IPV4/IPV6 flows using TCP/UDP
# IPFlows_withoutPorts = {}  # stores IPV4/IPV6 flows NOT using TCP/UDP
# EthFlows = {}  # stores flows NOT using IPV4/IPV6

print('Running for IPFlows_withPorts:')
print('\tnumber of MUDflows: ', len(IPFlows_withPorts))

# mapping the IPFlows_withPorts
for IPFlow_withPorts in IPFlows_withPorts:
    etherType = IPFlow_withPorts.split('-')[0]  # ipv4 OR ipv6
    ipProto = int(IPFlow_withPorts.split('-')[1])  # 6 OR 17
    port = int(IPFlow_withPorts.split('-')[2])  # port number
    # print('------------------------------------------------------------------')
    # print(etherType)
    # print(ipProto)
    # print(port)
    for fdcp in all_FDCPs:
        # checking if the FDCP has ether type of ipv4 or ipv6
        if ( (etherType == 'ipv4') and ("0x0800" in fdcp['fpd:metadata']["ether-types"]) ) or ( (etherType == 'ipv6') and ("0x86DD" in fdcp['fpd:metadata']["ether-types"]) ):
            # checking the IP protocol and port
            if ( ipProto in fdcp['fpd:metadata']["ip-protocols"] ) and  ( port in fdcp['fpd:metadata']["server-port"]["port"] ):
                # initializing an empty list if this is the first match
                if IPFlow_withPorts not in MUDFlowtoFDCP:
                    MUDFlowtoFDCP[IPFlow_withPorts] = []
                # adding the found match
                MUDFlowtoFDCP[IPFlow_withPorts].append(fdcp["fpd:info"]["abbreviation"]) 

    # checking whether there is no matching FDCP for the mud flow
    if IPFlow_withPorts not in MUDFlowtoFDCP:
        MUDFlowWithNoFDCP.append(IPFlow_withPorts)

# print('\tmapping:', MUDFlowtoFDCP)
numOfMaps_IPFlows_withPorts = len(MUDFlowtoFDCP)
print('\tnumber of maps:', numOfMaps_IPFlows_withPorts)
print('\n------------------------------------------------------------------\n')
#####################################################################################################################################################################

print('Running for IPFlows_withoutPorts:')
print('\tnumber of MUDflows: ', len(IPFlows_withoutPorts))
print('\tMUDflows:', IPFlows_withoutPorts)
# print(len(IPFlows_withoutPorts))
# print(len(EthFlows))
# mapping the IPFlows_withPorts
for IPFlow_withoutPorts in IPFlows_withoutPorts:
    etherType = IPFlow_withoutPorts.split('-')[0]  # ipv4 OR ipv6
    ipProto = int(IPFlow_withoutPorts.split('-')[1])  # ip proto number EXCEPT 6 OR 17
    
    # print(etherType)
    # print(ipProto)
    # print(port)
    for fdcp in all_FDCPs:
        # checking if the FDCP has ether type of ipv4 or ipv6
        if ( (etherType == 'ipv4') and ("0x0800" in fdcp['fpd:metadata']["ether-types"]) ) or ( (etherType == 'ipv6') and ("0x86DD" in fdcp['fpd:metadata']["ether-types"]) ):
            # checking the IP protocol 
            if ( ipProto in fdcp['fpd:metadata']["ip-protocols"] ):
                # initializing an empty list if this is the first match
                if IPFlow_withoutPorts not in MUDFlowtoFDCP:
                    MUDFlowtoFDCP[IPFlow_withoutPorts] = []
                # adding the found match
                MUDFlowtoFDCP[IPFlow_withoutPorts].append(fdcp["fpd:info"]["abbreviation"])
    
    # checking whether there is no matching FDCP for the mud flow
    if IPFlow_withoutPorts not in MUDFlowtoFDCP:
        MUDFlowWithNoFDCP.append(IPFlow_withoutPorts)

# print('\tmapping:', MUDFlowtoFDCP)
numOfMaps_IPFlows_withoutPorts = len(MUDFlowtoFDCP) - numOfMaps_IPFlows_withPorts  
print('\tnumber of maps:', numOfMaps_IPFlows_withoutPorts)
print('\n------------------------------------------------------------------\n')

#####################################################################################################################################################################

print('Running for EthFlows:')
print('\tnumber of MUDflows: ', len(EthFlows))
print('\tMUDflows:', EthFlows)
# print(len(IPFlows_withoutPorts))
# print(len(EthFlows))
# mapping the IPFlows_withPorts
for EthFlow in EthFlows:
    etherType = EthFlow.split('-')[1]  # ether type EXCEPT ipv4 OR ipv6
    
    # print(etherType)
    # print(ipProto)
    # print(port)
    for fdcp in all_FDCPs:
        # checking if the FDCP has the matching ether type
        for et in fdcp['fpd:metadata']["ether-types"]:
            ethMatch = False
            # handling an ether-type range
            if '-' in et:
                lowRange = et.split('-')[0]
                uppRange = et.split('-')[1]
                if int(etherType, base=16) in range(int(lowRange, base=16), int(uppRange, base=16) + 1):
                    ethMatch = True
                    
            # handling a single ether type value
            else:
                if et == etherType:
                    ethMatch = True
            
            if ethMatch:
                # initializing an empty list if this is the first match
                if EthFlow not in MUDFlowtoFDCP:
                    MUDFlowtoFDCP[EthFlow] = []
                # adding the found match
                MUDFlowtoFDCP[EthFlow].append(fdcp["fpd:info"]["abbreviation"])
    
    # checking whether there is no matching FDCP for the mud flow
    if EthFlow not in MUDFlowtoFDCP:
        MUDFlowWithNoFDCP.append(EthFlow)

# print('\tmapping:', MUDFlowtoFDCP)
numOfMaps_EthFlows  = len(MUDFlowtoFDCP) - numOfMaps_IPFlows_withPorts - numOfMaps_IPFlows_withoutPorts
print('\tnumber of maps:', numOfMaps_EthFlows)
print('\n------------------------------------------------------------------\n')

#####################################################################################################################################################################

print('total number of MUDflows:', len(IPFlows_withPorts) + len(IPFlows_withoutPorts) + len(EthFlows))
print('total number of MUDflows with FDCP:', numOfMaps_IPFlows_withPorts + numOfMaps_IPFlows_withoutPorts + numOfMaps_EthFlows)
print('number of MUDFlowWithNoFDCP:', len(MUDFlowWithNoFDCP))
print('\nMUDFlowWithNoFDCP:', MUDFlowWithNoFDCP)

Running for IPFlows_withPorts:
	number of MUDflows:  31
	number of maps: 13

------------------------------------------------------------------

Running for IPFlows_withoutPorts:
	number of MUDflows:  4
	MUDflows: {'ipv4-2-*': ['amazonEcho', 'tplinkcamera', 'withingssleepsensor', 'samsungsmartcam'], 'ipv4-1-*': ['amazonEcho'], 'ipv6-58-*': ['lifxbulb', 'nestsmokesensor', 'pixstarphotoframe', 'tribyspeaker', 'samsungsmartcam'], 'ipv6-0-*': ['nestsmokesensor', 'pixstarphotoframe', 'tribyspeaker', 'samsungsmartcam']}
	number of maps: 3

------------------------------------------------------------------

Running for EthFlows:
	number of MUDflows:  2
	MUDflows: {'ethernet-0x0006': ['amazonEcho', 'awairAirQuality', 'lifxbulb', 'nestsmokesensor', 'pixstarphotoframe', 'ringdoorbell', 'tplinkcamera', 'tribyspeaker', 'samsungsmartcam'], 'ethernet-0x888e': ['amazonEcho', 'awairAirQuality', 'lifxbulb', 'nestsmokesensor', 'pixstarphotoframe', 'ringdoorbell', 'tplinkcamera', 'tribyspeaker', 'withing

#### Writing the info to csv files

In [21]:
# writing dictionaries to store devices classified by MUD flows
# dict = {"MUDFlow" : [list of devices using the MUDFlow]}
#       IPFlows_withPorts = {}  # stores IPV4/IPV6 flows using TCP/UDP
#       IPFlows_withoutPorts = {}  # stores IPV4/IPV6 flows NOT using TCP/UDP
#       EthFlows = {}  # stores flows NOT using IPV4/IPV6

csvPath = '../../results/MUD/classifyDevicesByMUDFlows/'
fileNames = [IPFlows_withPorts, IPFlows_withoutPorts, EthFlows]
csvHeader = 'MUDFlow,Using_Devices\n'
for fileName in fileNames:
    if fileName == IPFlows_withPorts:
        csvFile = "IPFlows_withPorts"
    elif fileName == IPFlows_withoutPorts:
        csvFile = "IPFlows_withoutPorts"
    elif fileName == EthFlows:
        csvFile = "EthFlows"

    with open(csvPath + csvFile + '.csv', 'w') as f:
        f.write(csvHeader)
        for key in fileName.keys():
            f.write("%s,%s\n"%(key,str(fileName[key]).replace('[', '').replace(']', '').replace(',', ' |')))

In [22]:
# writing variable to hold the FDCP-MUDFlow mapping
#       MUDFlowtoFDCP = {}  # {'mudFlow': [corresponding FDCP abbreviations]}
#       MUDFlowWithNoFDCP = [] 


# writing MUDFlowtoFDCP
csvPath = '../../results/MUD/MUD_FDCP_map/'
fileName = 'MUDFlowtoFDCP'
csvHeader = 'MUDFlow,FDCPs\n'
with open(csvPath + fileName + '.csv', 'w') as f:
        f.write(csvHeader)
        for key in MUDFlowtoFDCP.keys():
            f.write("%s,%s\n"%(key,str(MUDFlowtoFDCP[key]).replace('[', '').replace(']', '').replace(',', ' |')))


# writing MUDFlowWithNoFDCP
csvPath = '../../results/MUD/MUD_FDCP_map/'
fileName = 'MUDFlowWithNoFDCP'
with open(csvPath + fileName + '.csv', 'w') as f:
        for el in MUDFlowWithNoFDCP:
            f.write("%s\n"%(el))

In [23]:
# writing dictionary to store MUD flows classified by devices
# dict = {"device" : [list of MUDFlows used by the device]}
#       devices_MUDFlows = {}
csvPath = '../../results/MUD/classifyMUDFlowsByDevices/'
fileName = 'devices_MUDFlows'
csvHeader = 'Device,Flows_Used\n'
with open(csvPath + fileName + '.csv', 'w') as f:
        f.write(csvHeader)
        for key in devices_MUDFlows.keys():
            f.write("%s,%s\n"%(key,str(devices_MUDFlows[key]).replace('[', '').replace(']', '').replace(',', ' |')))

In [24]:
print(int('0x0800', base=16))
print(int('0x080A', base=16) in range(int('0x0800', base=16), int('0x080A', base=16) +1))


2048
True
